# Purpose: Pig Analysis Segmentation and Specific Features

### Purpose: To move from thresholding to segmentation and shape feature quantification

Created by: Hawley Helmbrecht

Creation Date: 05/21/2021

Last Update: 06/4/2021 (updated to only include the Li threshold information)

*Step 1: Import Necessary Packages*

In [ ]:
import numpy as np
import pandas as pd
from scipy import ndimage

import skimage.filters
from skimage import morphology
from skimage.measure import label, regionprops, regionprops_table
from skimage.color import label2rgb
from skimage import io

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

*Step 2: User Inputs*

In [ ]:
#replace the example path from my computer with the path to the image on your computer

cell_im_location = '/Users/hhelmbre/Desktop/Sample_piglet_dataset/FGR_P4_2414_frontal_cortex_2.tif'

*Step 3: Reading in the Image*

In [ ]:
cell_im = io.imread(cell_im_location)

In [ ]:
cell_im.shape

*Step 4: Viewing the Image*

*Step 5: Splitting Channels for Thresholding*

In [ ]:
nucleus_im = cell_im[0,:,:] 
cell_im = cell_im[1,:,:]

In [ ]:
plt.imshow(cell_im)

*Step 6: Applying the Li Threshold*

In [ ]:
thresh_li = skimage.filters.threshold_li(cell_im)
binary_li = cell_im > thresh_li

*Step 7: Checking our Threshold*

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(10, 10))

ax[0, 0].imshow(cell_im, cmap=plt.cm.gray)
ax[0, 0].set_title('Original')

ax[0, 1].hist(cell_im.ravel(), bins=256)
ax[0, 1].set_title('Histogram')
ax[0, 1].set_xlim((0, 256)) 

ax[1, 0].imshow(binary_li, cmap=plt.cm.gray)
ax[1, 0].set_title('Thresholded (Otsu)')

ax[1, 1].hist(cell_im.ravel(), bins=256)
ax[1, 1].axvline(thresh_li, color='r')
ax[1, 1].set_xlim((0, 256)) 

for a in ax[:, 0]:
    a.axis('off')
plt.show()

#I do not know why the ravel has stoped working here

*Step 8: Removing Small Objects from the Threshold (Li) Image*

In [ ]:
thresh_li = skimage.filters.threshold_li(cell_im)
binary_li = cell_im > thresh_li

In [ ]:
new_binary_li = morphology.remove_small_objects(binary_li, min_size=64)

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(10, 10))

ax[0, 0].imshow(cell_im, cmap=plt.cm.gray)
ax[0, 0].set_title('Original')

ax[0, 1].hist(cell_im.ravel(), bins=256)
ax[0, 1].set_title('Histogram')
ax[0, 1].set_xlim((0, 256)) 

ax[1, 0].imshow(new_binary_li, cmap=plt.cm.gray)
ax[1, 0].set_title('Thresholded (Otsu)')

ax[1, 1].hist(cell_im.ravel(), bins=256)
ax[1, 1].axvline(thresh_otsu, color='r')
ax[1, 1].set_xlim((0, 256)) 

for a in ax[:, 0]:
    a.axis('off')
plt.show()

#Still not sure why the ravel is not working

*Step 9: Labeling the Image*

In [ ]:
label_image = label(new_binary_li)
image_label_overlay = label2rgb(label_image, image=new_binary_li, bg_label=0)

*Step 10: Viewing the labeled image with area boxes*

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
ax.imshow(image_label_overlay)

for region in regionprops(label_image):
    # take regions with large enough areas
    if region.area >= 100:
        # draw rectangle around segmented coins
        minr, minc, maxr, maxc = region.bbox
        rect = mpatches.Rectangle((minc, minr), maxc - minc, maxr - minr,
                                  fill=False, edgecolor='red', linewidth=2)
        ax.add_patch(rect)

ax.set_axis_off()
plt.tight_layout()
plt.show()

*Step 10: Filling in shape holes to see if it improves our labeling*

In [ ]:

new_binary_otsu = ndimage.binary_fill_holes(new_binary_li)
new_binary_otsu = morphology.remove_small_objects(new_binary_li, 500)

In [ ]:
label_image = label(new_binary_otsu)
image_label_overlay = label2rgb(label_image, image=new_binary_otsu, bg_label=0)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
ax.imshow(image_label_overlay)

for region in regionprops(label_image):
    # take regions with large enough areas
    if region.area >= 500:
        # draw rectangle around segmented coins
        minr, minc, maxr, maxc = region.bbox
        rect = mpatches.Rectangle((minc, minr), maxc - minc, maxr - minr,
                                  fill=False, edgecolor='red', linewidth=2)
        ax.add_patch(rect)

ax.set_axis_off()
plt.tight_layout()
plt.show()

*Step 12: Getting a .csv file of multiple regionprops*

In [ ]:
from skimage import measure 

props = measure.regionprops_table(label_image, properties=('perimeter',
                                                 'area',
                                                 'major_axis_length',
                                                 'minor_axis_length',))

In [ ]:
green_shape_features = pd.DataFrame(props)

*Step 13: Viewing the Table*

In [ ]:
green_shape_features

*Step 14: Caculating the Circularity*

In [ ]:
green_shape_features['circularity'] = 4*np.pi*green_shape_features.area/green_shape_features.perimeter**2

In [ ]:
green_shape_features

*Step 15: Calculating the Aspect Ratio*

In [ ]:
green_shape_features['aspect_ratio'] = green_shape_features.major_axis_length/green_shape_features.minor_axis_length

In [ ]:
green_shape_features

*Step 16: Plotting some values*

In [ ]:
green_shape_features['stain'] = 'iba1'

In [ ]:
green_shape_features.plot(x ='perimeter', y='area', kind = 'scatter')

*Step 17: Saving as a CSV file*

In [ ]:
green_shape_features.to_csv('/Users/hhelmbre/Desktop/Sample_piglet_dataset/FGR_P4_2414_frontal_cortex_shape_features.csv')

*Step 18: Individual Exploration*

Apply these steps to a different stain, try to add new features from region props, try different plotting methods in the notebook, take the CSV and do some plotting of your own!

Next Week: We will get into processing multiple images and into experimental treatment groups. 